# Wiki NBA Stat Scraper

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
url = 'https://en.wikipedia.org/wiki/Michael_Jordan'

In [ ]:
# Check to see if this works on other players
# url = 'https://en.wikipedia.org/wiki/Kobe_Bryant'

In [ ]:
# Some of the wiki pages for players have a different format to 
# scrape from. Use this line for pages like this:
# url = 'https://en.wikipedia.org/wiki/Steve_Nash'
# or
# url = 'https://en.wikipedia.org/wiki/Shawn_Kemp'


# Here is the line to use

# Maybe write a try execpt for the different formats.


# wikitable = soup.find('table', {'class':'wikitable'})

In [3]:
r = requests.get(url)
text = r.text
soup = BeautifulSoup(text, 'html.parser')

In [4]:
# Check to make sure you have a connection to the website
r.status_code

200

In [ ]:
# Use prettify to get a good
# look at the HTML
# print(soup.prettify())

In [5]:
# I'm using this method to create wikitable 
wikitable = soup.find('table', {'class':'wikitable sortable'})

In [ ]:
# wikitable = soup.find('table', {'class':'wikitable'})

In [6]:
# Find all 'th' and 'td'
th = wikitable.find_all(['th'])
td = wikitable.find_all(['td'])

In [ ]:
# th # delete this later

In [ ]:
# td # delete this later

In [7]:
stat_header = []
for col in th:
    col = col.get_text()
    col = col.rstrip()
    stat_header.append(col)

In [ ]:
# stat_header # delete this later

In [8]:
stat = []
for row in td:
    row = row.get_text()
    row = row.rstrip()
    stat.append(row)

In [9]:
# Drop the last two rows starting @ Career
# These rows have a different shape 
# and will be a problem in merging the data
x = True
regular_season = []
while x == True:
    for s in stat:
        if s != 'Career':
            regular_season.append(s)
            continue
        else:
            x = False
            break

In [10]:
# This line creates a list that matches 
# the lenght of the regular season stats
header = stat_header * (len(regular_season)//len(stat_header))

In [11]:
# Creates a list of tuples
# Unpacked the to list into tuple to save data
# Important to keep the header column and stat (key/value) pair connected
result = list(zip(header, regular_season))

In [ ]:
#result # delete later

In [12]:
sYear = []
sTeam = []
sGP = []
sGS = []
sMPG = []
sFG = []
s3P = []
sFT = []
sRPG = []
sAPG = []
sSPG = []
sBPG = []
sPPG = []

In [13]:
# loop through the results list of tuples 
# and add the stats to the correct list
for x in result:
    if x[0] == 'Year':
        sYear.append(x[1])
    elif x[0] == 'Team':
        sTeam.append(x[1])
    elif x[0] == 'GP':
        sGP.append(x[1])
    elif x[0] == 'GS':
        sGS.append(x[1])
    elif x[0] == 'MPG':
        sMPG.append(x[1])
    elif x[0] == 'FG%':
        sFG.append(x[1])
    elif x[0] == '3P%':
        s3P.append(x[1])
    elif x[0] == 'FT%':
        sFT.append(x[1])
    elif x[0] == 'RPG':
        sRPG.append(x[1])
    elif x[0] == 'APG':
        sAPG.append(x[1])
    elif x[0] == 'SPG':
        sSPG.append(x[1])
    elif x[0] == 'BPG':
        sBPG.append(x[1])
    elif x[0] == 'PPG':
        sPPG.append(x[1])

In [14]:
# Now that the stats are in the correct 
# list create the dataframe
df1 = pd.DataFrame(list(zip(sYear,sTeam,sGP,sGS,sMPG,sFG,s3P,sFT,sRPG,sAPG,sSPG,sBPG,sPPG)), 
                   columns =['Year', 'Team', 'GP','GS','MPG','FG%','3P%','FT%','RPG','APG','SPG','BPG','PPG']) 

In [15]:
# All the successfully scraped from the website and now sits in this dataframe
df1

,Year,Team,GP,GS,MPG,FG%,3P%,FT%,RPG,APG,SPG,BPG,PPG
0,1984–85,Chicago,82,82,38.3,.515,.173,.845,6.5,5.9,2.4,.8,28.2
1,1985–86,Chicago,18,7,25.1,.457,.167,.840,3.6,2.9,2.1,1.2,22.7
2,1986–87,Chicago,82,82,40.0,.482,.182,.857,5.2,4.6,2.9,1.5,37.1*
3,1987–88,Chicago,82,82,40.4*,.535,.132,.841,5.5,5.9,3.2*,1.6,35.0*
4,1988–89,Chicago,81,81,40.2*,.538,.276,.850,8.0,8.0,2.9,.8,32.5*
5,1989–90,Chicago,82,82,39.0,.526,.376,.848,6.9,6.3,2.8*,.7,33.6*
6,1990–91†,Chicago,82,82,37.0,.539,.312,.851,6.0,5.5,2.7,1.0,31.5*
7,1991–92†,Chicago,80,80,38.8,.519,.270,.832,6.4,6.1,2.3,.9,30.1*
8,1992–93†,Chicago,78,78,39.3,.495,.352,.837,6.7,5.5,2.8*,.8,32.6*
9,1994–95,Chicago,17,17,39.3,.411,.500,.801,6.9,5.3,1.8,.8,26.9


In [ ]:
# Repeat the same for the playoff numbers

In [ ]:
# Upload in SQLite
# If play is in db do nothing
# Else upload data

In [ ]:
# Create visualizations in matplotlib